In [1]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install transformers
!pip install datasets
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!nvidia-smi

Thu Sep 22 13:21:36 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
import os
import gc
import math
import time
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter('ignore')
from tqdm import tqdm
import re
import html

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import Adam, SGD, AdamW, RAdam
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, Dataset

from sklearn.model_selection import StratifiedKFold,StratifiedGroupKFold,GroupKFold
from sklearn.metrics import log_loss,f1_score
from sklearn.feature_extraction.text import TfidfVectorizer

from transformers import AutoModel, AutoConfig, AutoTokenizer, AdamW, DataCollatorWithPadding
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import lightgbm as lgb
import catboost as ctb

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
seed=42

In [6]:
DIR = '/content/drive/MyDrive/Competitions/Signate/MUFJ'
INPUT_DIR = os.path.join(DIR,'input')
OUTPUT_DIR = os.path.join(DIR,'output')
OUTPUT_SUB_DIR = os.path.join(OUTPUT_DIR,'submission')
#OUTPUT_MODEL_DIR = os.path.join(OUTPUT_DIR,'model')
#OUTPUT_MODEL_DIR19 = DIR + '/output/model/EXP19/'
OUTPUT_MODEL_DIR20 = DIR + '/output/model/EXP20/'
#OUTPUT_MODEL_DIR22 = DIR + '/output/model/EXP22/'
#OUTPUT_MODEL_DIR23 = DIR + '/output/model/EXP23/'
#OUTPUT_MODEL_DIR24 = DIR + '/output/model/EXP24/'
#OUTPUT_MODEL_DIR25 = DIR + '/output/model/EXP25/'
#OUTPUT_MODEL_DIR27 = DIR + '/output/model/EXP27/'
#OUTPUT_MODEL_DIR28 = DIR + '/output/model/EXP28/'
#OUTPUT_MODEL_DIR31 = DIR + '/output/model/EXP31/'
#OUTPUT_MODEL_DIR41 = DIR + '/output/model/EXP41/'
OUTPUT_MODEL_DIR42 = DIR + '/output/model/EXP42/'

#OUTPUT_MODEL_DIR21 = DIR + '/output/model/EXP21/'
#OUTPUT_MODEL_DIR43 = DIR + '/output/model/EXP43/'
OUTPUT_MODEL_DIR44 = DIR + '/output/model/EXP44/'
OUTPUT_MODEL_DIR45 = DIR + '/output/model/EXP45/'
#OUTPUT_MODEL_DIR46 = DIR + '/output/model/EXP46/'

OUTPUT_MODEL_DIR34 = DIR + '/output/model/EXP34/'
#OUTPUT_MODEL_DIR39 = DIR + '/output/model/EXP39/'
#OUTPUT_MODEL_DIR40 = DIR + '/output/model/EXP40/'
OUTPUT_MODEL_DIR47 = DIR + '/output/model/EXP47/'
OUTPUT_MODEL_DIR48 = DIR + '/output/model/EXP48/'

In [7]:
class CFG20:
    num_workers=4
    path=OUTPUT_MODEL_DIR20
    config_path=path+'config.pth'
    model="microsoft/deberta-v3-large"
    batch_size=32
    fc_dropout=0.2
    target_size=1
    max_len=256
    seed=42
    n_fold=4
    trn_fold=[0, 1, 2, 3]
    gradient_checkpointing=True
    freezing=True
    clean_content = True
    target2col = False



class CFG42:
    num_workers=4
    path=OUTPUT_MODEL_DIR42
    config_path=path+'config.pth'
    model="facebook/bart-large-mnli"
    batch_size=32
    fc_dropout=0.2
    target_size=1
    max_len=180
    seed=42
    n_fold=4
    trn_fold=[0, 1, 2, 3]
    gradient_checkpointing=False
    freezing=False
    clean_content = True
    target2col = False



class CFG44:
    num_workers=4
    path=OUTPUT_MODEL_DIR44
    config_path=path+'config.pth'
    model="microsoft/deberta-v3-large"
    batch_size=32
    fc_dropout=0.2
    target_size=1
    max_len=256
    seed=42
    n_fold=4
    trn_fold=[0, 1, 2, 3]
    gradient_checkpointing=False
    freezing=True
    clean_content = True
    target2col = False

class CFG45:
    num_workers=4
    path=OUTPUT_MODEL_DIR45
    config_path=path+'config.pth'
    model="facebook/bart-large-mnli"
    batch_size=32
    fc_dropout=0.2
    target_size=1
    max_len=180
    seed=42
    n_fold=4
    trn_fold=[0, 1, 2, 3]
    gradient_checkpointing=False
    freezing=False
    clean_content = False
    target2col = False



class CFG34:
    num_workers=4
    path=OUTPUT_MODEL_DIR34
    config_path=path+'config.pth'
    model="microsoft/deberta-v3-base"
    batch_size=32
    fc_dropout=0.2
    target_size=2
    max_len=256
    seed=42
    n_fold=4
    trn_fold=[0, 1, 2, 3]
    gradient_checkpointing=True
    freezing=True
    clean_content = True
    target_cols = ["state","category1"]
    target2col = True



class CFG47:
    num_workers=4
    path=OUTPUT_MODEL_DIR47
    config_path=path+'config.pth'
    model="funnel-transformer/large"
    batch_size=32
    fc_dropout=0.2
    target_size=2
    max_len=256
    seed=42
    n_fold=4
    trn_fold=[0, 1, 2, 3]
    gradient_checkpointing=False
    freezing=False
    clean_content = True
    target_cols = ["state","category1"]
    target2col = True

class CFG48:
    num_workers=4
    path=OUTPUT_MODEL_DIR48
    config_path=path+'config.pth'
    model="facebook/bart-large-mnli"
    batch_size=32
    fc_dropout=0.2
    target_size=2
    max_len=180
    seed=42
    n_fold=4
    trn_fold=[1, 2, 3]
    gradient_checkpointing=False
    freezing=False
    clean_content = True
    target_cols = ["state","category1"]
    target2col = True

In [8]:
def get_score(labels, outputs):
    thresh = 0.5
    y_pred = outputs
    y_true = labels
    return f1_score(y_true, (y_pred>thresh).astype(int))


def get_logger(filename=OUTPUT_DIR+'/infrence'):
    from logging import getLogger, INFO, FileHandler, Formatter, StreamHandler
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=f"{filename}.log")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = get_logger()

def seed_everything(seed=seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=seed)

In [9]:
def freeze(module):
    """
    Freezes module's parameters.
    """
    
    for parameter in module.parameters():
        parameter.requires_grad = False
        
def get_freezed_parameters(module):
    """
    Returns names of freezed parameters of the given module.
    """
    
    freezed_parameters = []
    for name, parameter in module.named_parameters():
        if not parameter.requires_grad:
            freezed_parameters.append(name)
            
    return freezed_parameters

def set_embedding_parameters_bits(embeddings_path, optim_bits=32):
    """
    https://github.com/huggingface/transformers/issues/14819#issuecomment-1003427930
    """
    
    embedding_types = ("word", "position", "token_type")
    for embedding_type in embedding_types:
        attr_name = f"{embedding_type}_embeddings"
        
        if hasattr(embeddings_path, attr_name): 
            bnb.optim.GlobalOptimManager.get_instance().register_module_override(
                getattr(embeddings_path, attr_name), 'weight', {'optim_bits': optim_bits}
            )

In [10]:
#oof_19 = pd.read_pickle(CFG19.path+'oof_df.pkl')
oof_20 = pd.read_pickle(CFG20.path+'oof_df.pkl')
#oof_22 = pd.read_pickle(CFG22.path+'oof_df.pkl')
#oof_23 = pd.read_pickle(CFG23.path+'oof_df.pkl')
#oof_24 = pd.read_pickle(CFG24.path+'oof_df.pkl')
#oof_25 = pd.read_pickle(CFG25.path+'oof_df.pkl')
#oof_27 = pd.read_pickle(CFG27.path+'oof_df.pkl')
#oof_28 = pd.read_pickle(CFG28.path+'oof_df.pkl')
#oof_31 = pd.read_pickle(CFG31.path+'oof_df.pkl')
#oof_41 = pd.read_pickle(CFG41.path+'oof_df.pkl')
oof_42 = pd.read_pickle(CFG42.path+'oof_df.pkl')

#oof_21 = pd.read_pickle(CFG21.path+'oof_df.pkl')
#oof_43 = pd.read_pickle(CFG43.path+'oof_df.pkl')
oof_44 = pd.read_pickle(CFG44.path+'oof_df.pkl')
oof_45 = pd.read_pickle(CFG45.path+'oof_df.pkl')
#oof_46 = pd.read_pickle(CFG46.path+'oof_df.pkl')

oof_34 = pd.read_pickle(CFG34.path+'oof_df.pkl')
#oof_39 = pd.read_pickle(CFG39.path+'oof_df.pkl')
#oof_40 = pd.read_pickle(CFG40.path+'oof_df.pkl')
oof_47 = pd.read_pickle(CFG47.path+'oof_df.pkl')
oof_48 = pd.read_pickle(CFG48.path+'oof_df.pkl')

In [11]:
oof_df = pd.DataFrame()

oof_df["id"] = oof_20["id"].values
oof_df["kfold"] = oof_20["kfold"].values
oof_df["state"] = oof_20['state'].values

#oof_df["preds_19"] = oof_19['pred'].values
oof_df["preds_20"] = oof_20['pred'].values
#oof_df["preds_22"] = oof_22['pred'].values
#oof_df["preds_23"] = oof_23['pred'].values
#oof_df["preds_24"] = oof_24['pred'].values
#oof_df["preds_25"] = oof_25['pred'].values
#oof_df["preds_27"] = oof_27['pred'].values
#oof_df["preds_28"] = oof_28['pred'].values
#oof_df["preds_31"] = oof_31['pred'].values
#oof_df["preds_41"] = oof_41['pred'].values
oof_df["preds_42"] = oof_42['pred'].values

#oof_df["preds_21"] = oof_21['pred'].values
#oof_df["preds_43"] = oof_43['pred'].values
oof_df["preds_44"] = oof_44['pred'].values
oof_df["preds_45"] = oof_45['pred'].values
#oof_df["preds_46"] = oof_46['pred'].values

oof_df["preds_34"] = oof_34["pred_state"].values
#oof_df["preds_39"] = oof_39["pred_state"].values
#oof_df["preds_40"] = oof_40["pred_state"].values
oof_df["preds_47"] = oof_47["pred_state"].values
oof_df["preds_48"] = oof_48["pred_state"].values

oof_df["preds_max"] = oof_df.loc[:,"preds_20":"preds_48"].max(axis=1)
oof_df["preds_min"] = oof_df.loc[:,"preds_20":"preds_48"].min(axis=1)
oof_df["preds_mean"] = oof_df.loc[:,"preds_20":"preds_48"].mean(axis=1)
oof_df["preds_std"] = oof_df.loc[:,"preds_20":"preds_48"].std(axis=1)

In [12]:
features = oof_df.columns.to_list()
features.remove('id')
features.remove('state')
features.remove('kfold')

In [13]:
features = features + ['country_category3_success_rate','prod_goal_duration','punct_percent','mean_word_len','ratio_goal_duration','count_sent','num_words_title','content_sum','content_max','content_mean']

In [14]:
train_df = pd.read_csv(os.path.join(OUTPUT_DIR,"feature_train.csv"))
test_df = pd.read_csv(os.path.join(OUTPUT_DIR,"feature_test.csv"))

p = re.compile(r"<[^>]*?>|&amp;|[/'’\"”]")
train_df['html_content'] = train_df['html_content'].map(lambda x: str(x))
train_df['html_content'] = train_df['html_content'].apply(html.unescape)
train_df['html_content'] = train_df['html_content'].map(lambda x: p.sub("", x))
train_df['html_content'] = train_df['html_content'].map(lambda x: x.lstrip())
train_df['html_content'] = train_df['html_content'].fillna('missing')

test_df['html_content'] = test_df['html_content'].map(lambda x: str(x))
test_df['html_content'] = test_df['html_content'].apply(html.unescape)
test_df['html_content'] = test_df['html_content'].map(lambda x: p.sub("", x))
test_df['html_content'] = test_df['html_content'].map(lambda x: x.lstrip())
test_df['html_content'] = test_df['html_content'].fillna('missing')

vec_tfidf = TfidfVectorizer(stop_words=None,min_df=8,binary=False,dtype=np.float32)
train_tfidf = vec_tfidf.fit_transform(train_df['html_content'].fillna('missing').str.lower().values)
test_tfidf = vec_tfidf.transform(test_df['html_content'].fillna('missing').str.lower().values)

train_df["content_sum"] = np.array(train_tfidf.sum(axis=1))[:,0]
train_df["content_max"] = np.array(train_tfidf.max(axis=1).todense())[:,0]
train_df["content_mean"] = train_df["content_sum"] /train_tfidf.getnnz(axis=1)

test_df["content_sum"] = np.array(test_tfidf.sum(axis=1))[:,0]
test_df["content_max"] = np.array(test_tfidf.max(axis=1).todense())[:,0]
test_df["content_mean"] = test_df["content_sum"] /test_tfidf.getnnz(axis=1)


train_df = train_df.fillna(0)
test_df = test_df.fillna(0)

oof_df = oof_df.merge(train_df,how="left",on=["id","state"])

In [15]:
#preds_features = oof_df.loc[:,"preds_19":"preds_48"].columns.to_list()
#for col in preds_features:
#  oof_df[f"{col}_label"] = (oof_df[col]>0.5).astype(int)
#oof_df["label"] = oof_df.loc[:,"preds_19_label":"preds_48_label"].mode(axis=1)[0]
#mode_score = f1_score(oof_df.state, oof_df.label)
#print(f"mode score : {mode_score}")

In [16]:
oof_df

,id,kfold,state,preds_20,preds_42,preds_44,preds_45,preds_34,preds_47,preds_48,...,country_category1_ratio_goal_durataion_max,country_category1_ratio_goal_durataion_mean,country_category1_ratio_goal_durataion_std,country_category1_prod_goal_durataion_min,country_category1_prod_goal_durataion_max,country_category1_prod_goal_durataion_mean,country_category1_prod_goal_durataion_std,content_sum,content_max,content_mean
0,train_00002,0,0,0.842587,0.687663,0.039490,0.618032,0.562053,0.127907,0.497112,...,12500.000000,244.482430,578.729051,500,6000000,249136.882129,5.500174e+05,11.562983,0.279128,0.060224
1,train_00006,0,1,0.973275,0.975800,0.983077,0.944546,0.863617,0.998426,0.945155,...,1976.190476,161.228066,278.825054,500,1605000,163663.461538,2.433703e+05,8.527903,0.304559,0.087019
2,train_00016,0,0,0.000086,0.000005,0.000125,0.000720,0.005788,0.000015,0.144979,...,3083.333333,383.636249,716.542518,2000,6000000,430190.476190,1.003638e+06,14.089891,0.318688,0.041810
3,train_00017,0,1,0.039410,0.968401,0.006329,0.819439,0.466576,0.995058,0.289448,...,5882.352941,618.340636,764.186355,1500,6000000,736182.698515,1.047290e+06,6.702881,0.480725,0.111715
4,train_00018,0,1,0.157502,0.003776,0.987779,0.113744,0.696554,0.000591,0.118970,...,9214.285714,453.816242,908.876763,2500,6000000,446428.039702,8.592309e+05,11.283503,0.478463,0.025939
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9786,train_09768,3,0,0.987880,0.257114,0.937392,0.199166,0.405543,0.085813,0.277783,...,20000.000000,515.681657,899.536571,1000,6000000,594444.188428,1.054557e+06,7.724467,0.252752,0.108795
9787,train_09770,3,0,0.000626,0.004057,0.000465,0.001223,0.010382,0.001217,0.012598,...,5678.571429,253.998238,591.039423,1000,6000000,262606.382979,7.007054e+05,5.009840,0.414151,0.151813
9788,train_09778,3,1,0.999716,0.878590,0.999496,0.990713,0.952588,0.985981,0.982950,...,7950.000000,500.218037,857.122410,1500,6000000,566538.152610,1.064384e+06,8.366793,0.520718,0.051647
9789,train_09781,3,0,0.000817,0.072511,0.002549,0.201423,0.056492,0.015276,0.520327,...,5882.352941,618.340636,764.186355,1500,6000000,736182.698515,1.047290e+06,9.228383,0.309242,0.083139


In [17]:
def get_lgbm_cv_score(oof_df, lr=0.1, n_est=100, n_leave=31):
    f1_score_ = []
    clf_ = []
    _df = pd.DataFrame()
    for fold in range(4):
        print("Fold :", fold)
        train_dataset = oof_df.loc[oof_df.kfold!=fold].reset_index(drop=True)
        valid_dataset = oof_df.loc[oof_df.kfold==fold].reset_index(drop=True)
        y_train = train_dataset.state.values
        X_train = train_dataset[features].values
        y_test = valid_dataset.state.values
        X_test = valid_dataset[features].values

        clf = lgb.LGBMClassifier(learning_rate=lr,
                                n_estimators=n_est,
                                num_leaves=n_leave,
                                boosting_type='gbdt',
                                random_state=42)
        clf.fit(X_train, y_train, eval_set=(X_test, y_test), early_stopping_rounds=100,
               verbose=500)
        
        y_pred = clf.predict_proba(X_test)
        y_pred = y_pred[:,1]
        score = get_score(y_test, y_pred)
        valid_dataset["pred_lgb"] = y_pred
        _df = pd.concat([_df,valid_dataset])
        print(f"F1 Score : {score}")
        f1_score_.append(score)
        clf_.append(clf)
    return f1_score_, clf_,_df

lr=0.05
lgbm_f1_score_, lgbm_clf_,lgbm_df_ = get_lgbm_cv_score(oof_df, lr=lr, n_est=1000, n_leave=31)
print(np.array(lgbm_f1_score_).mean())
#0.8421830741466366

Fold : 0
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[87]	valid_0's binary_logloss: 0.32336
F1 Score : 0.842192006592501
Fold : 1
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[109]	valid_0's binary_logloss: 0.332896
F1 Score : 0.8362427265170408
Fold : 2
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[98]	valid_0's binary_logloss: 0.32992
F1 Score : 0.8430566967953985
Fold : 3
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[98]	valid_0's binary_logloss: 0.307795
F1 Score : 0.8532477947072975
0.8436848061530594


In [18]:
def get_ctb_cv_score(oof_df):
    params = {
          "loss_function":"Logloss",
          "n_estimators":1000,
          "random_seed":42,
          "learning_rate":0.05,
          #'task_type' : 'GPU'
    }
    f1_score_ = []
    clf_ = []
    _df = pd.DataFrame()
    for fold in range(4):
        print("Fold :", fold)
        train_dataset = oof_df.loc[oof_df.kfold!=fold].reset_index(drop=True)
        valid_dataset = oof_df.loc[oof_df.kfold==fold].reset_index(drop=True)
        y_train = train_dataset.state.values
        X_train = train_dataset[features].values
        y_test = valid_dataset.state.values
        X_test = valid_dataset[features].values

        clf = ctb.CatBoostClassifier(**params)
        clf.fit(X_train, y_train,
            eval_set=[(X_train,y_train),(X_test, y_test)],
            verbose = 500,
            early_stopping_rounds = 100)
        
        y_pred = clf.predict_proba(X_test)
        y_pred = y_pred[:,1]
        score = get_score(y_test, y_pred)
        valid_dataset["pred_ctb"] = y_pred
        _df = pd.concat([_df,valid_dataset])
        print(f"F1 Score : {score}")
        f1_score_.append(score)
        clf_.append(clf)       

    return f1_score_, clf_,_df


ctb_f1_score_, ctb_clf_,ctb_df_ = get_ctb_cv_score(oof_df)
print(np.array(ctb_f1_score_).mean())
#0.8466115173867548

Fold : 0
0:	learn: 0.6565091	test: 0.6565352	test1: 0.6573392	best: 0.6573392 (0)	total: 68.8ms	remaining: 1m 8s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.3219066323
bestIteration = 117

Shrink model to first 118 iterations.
F1 Score : 0.8485596707818929
Fold : 1
0:	learn: 0.6584845	test: 0.6585097	test1: 0.6604299	best: 0.6604299 (0)	total: 28.1ms	remaining: 28s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.3298977277
bestIteration = 149

Shrink model to first 150 iterations.
F1 Score : 0.8411910669975186
Fold : 2
0:	learn: 0.6573713	test: 0.6574013	test1: 0.6572206	best: 0.6572206 (0)	total: 14.8ms	remaining: 14.8s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.3326435072
bestIteration = 149

Shrink model to first 150 iterations.
F1 Score : 0.8470873786407767
Fold : 3
0:	learn: 0.6509244	test: 0.6509635	test1: 0.6524820	best: 0.6524820 (0)	total: 13.8ms	remaining: 13.8s
Stopped by overfitting detector  (100 itera

In [19]:
def get_logit_cv_score(oof_df):
    f1_score_ = []
    clf_ = []
    _df = pd.DataFrame()
    for fold in range(4):
        print("Fold :", fold)
        train_dataset = oof_df.loc[oof_df.kfold!=fold].reset_index(drop=True)
        valid_dataset = oof_df.loc[oof_df.kfold==fold].reset_index(drop=True)
        y_train = train_dataset.state.values
        X_train = train_dataset[features].values
        y_test = valid_dataset.state.values
        X_test = valid_dataset[features].values

        clf = LogisticRegression(random_state=seed)
        clf.fit(X_train, y_train)
        
        y_pred = clf.predict_proba(X_test)
        y_pred = y_pred[:,1]
        score = get_score(y_test, y_pred)
        valid_dataset["pred_logit"] = y_pred
        _df = pd.concat([_df,valid_dataset])
        print(f"F1 Score : {score}")
        f1_score_.append(score)
        clf_.append(clf) 
    return f1_score_, clf_,_df


logit_f1_score_, logit_clf_,logit_df_ = get_logit_cv_score(oof_df)
print(np.array(logit_f1_score_).mean())

Fold : 0
F1 Score : 0.6921634795650544
Fold : 1
F1 Score : 0.7089665653495442
Fold : 2
F1 Score : 0.7168850072780204
Fold : 3
F1 Score : 0.7190332326283988
0.7092620712052544


In [20]:
def get_svm_cv_score(oof_df):
    f1_score_ = []
    clf_ = []
    _df = pd.DataFrame()
    for fold in range(4):
        print("Fold :", fold)
        train_dataset = oof_df.loc[oof_df.kfold!=fold].reset_index(drop=True)
        valid_dataset = oof_df.loc[oof_df.kfold==fold].reset_index(drop=True)
        y_train = train_dataset.state.values
        X_train = train_dataset[features].values
        y_test = valid_dataset.state.values
        X_test = valid_dataset[features].values

        clf = SVC(C=5,kernel='rbf',gamma='auto',probability=True,random_state=seed)
        clf.fit(X_train, y_train)
        
        y_pred = clf.predict_proba(X_test)
        y_pred = y_pred[:,1]
        score = get_score(y_test, y_pred)
        valid_dataset["pred_svm"] = y_pred
        _df = pd.concat([_df,valid_dataset])
        print(f"F1 Score : {score}")
        f1_score_.append(score)
        clf_.append(clf) 
    return f1_score_, clf_,_df

svm_f1_score_, svm_clf_,svm_df_ = get_svm_cv_score(oof_df)
print(np.array(svm_f1_score_).mean())

Fold : 0
F1 Score : 0.6910817226178365
Fold : 1
F1 Score : 0.696459264352011
Fold : 2
F1 Score : 0.6862944162436548
Fold : 3
F1 Score : 0.6967830253251197
0.6926546071346555


In [21]:
staking_oof = lgbm_df_.merge(ctb_df_[["id","pred_ctb"]],how="left",on="id").merge(logit_df_[["id","pred_logit"]],how="left",on="id").merge(svm_df_[["id","pred_svm"]],how="left",on="id")
staking_oof.to_csv(os.path.join(OUTPUT_DIR,"staking_oof.csv"),index=False)
staking_oof

,id,kfold,state,preds_20,preds_42,preds_44,preds_45,preds_34,preds_47,preds_48,...,country_category1_prod_goal_durataion_max,country_category1_prod_goal_durataion_mean,country_category1_prod_goal_durataion_std,content_sum,content_max,content_mean,pred_lgb,pred_ctb,pred_logit,pred_svm
0,train_00002,0,0,0.842587,0.687663,0.039490,0.618032,0.562053,0.127907,0.497112,...,6000000,249136.882129,5.500174e+05,11.562983,0.279128,0.060224,0.480774,0.529852,0.540570,0.523663
1,train_00006,0,1,0.973275,0.975800,0.983077,0.944546,0.863617,0.998426,0.945155,...,1605000,163663.461538,2.433703e+05,8.527903,0.304559,0.087019,0.993731,0.979272,0.489914,0.910658
2,train_00016,0,0,0.000086,0.000005,0.000125,0.000720,0.005788,0.000015,0.144979,...,6000000,430190.476190,1.003638e+06,14.089891,0.318688,0.041810,0.010165,0.009230,0.579893,0.441579
3,train_00017,0,1,0.039410,0.968401,0.006329,0.819439,0.466576,0.995058,0.289448,...,6000000,736182.698515,1.047290e+06,6.702881,0.480725,0.111715,0.382605,0.458584,0.399798,0.334471
4,train_00018,0,1,0.157502,0.003776,0.987779,0.113744,0.696554,0.000591,0.118970,...,6000000,446428.039702,8.592309e+05,11.283503,0.478463,0.025939,0.490914,0.397390,0.718700,0.523316
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9786,train_09768,3,0,0.987880,0.257114,0.937392,0.199166,0.405543,0.085813,0.277783,...,6000000,594444.188428,1.054557e+06,7.724467,0.252752,0.108795,0.489307,0.539863,0.508474,0.609656
9787,train_09770,3,0,0.000626,0.004057,0.000465,0.001223,0.010382,0.001217,0.012598,...,6000000,262606.382979,7.007054e+05,5.009840,0.414151,0.151813,0.014895,0.007770,0.034910,0.522863
9788,train_09778,3,1,0.999716,0.878590,0.999496,0.990713,0.952588,0.985981,0.982950,...,6000000,566538.152610,1.064384e+06,8.366793,0.520718,0.051647,0.996549,0.983253,0.623150,0.754494
9789,train_09781,3,0,0.000817,0.072511,0.002549,0.201423,0.056492,0.015276,0.520327,...,6000000,736182.698515,1.047290e+06,9.228383,0.309242,0.083139,0.121573,0.180016,0.498686,0.083188


In [22]:
#oof_prediction = staking_oof["pred_lgb"]*1.40719207e-01 + staking_oof["pred_ctb"]*3.79171486e-02 + staking_oof["pred_logit"]*3.99166372e-05 + staking_oof["pred_svm"]*8.21323728e-01
oof_prediction = (staking_oof["pred_lgb"]+staking_oof['pred_ctb']) / 2
score = get_score(staking_oof.state, oof_prediction)
print(f"F1 Score : {score}")

F1 Score : 0.8458708094848731


In [23]:
best_score = 0
best_thresh = 0.5
y_pred = oof_prediction
y_true = staking_oof.state
for thresh in np.arange(0.2, 0.801, 0.01):
    thresh = np.round(thresh, 2)
    score = f1_score(y_true, (y_pred>thresh).astype(int))
    #print("F1 score at threshold {0} is {1}".format(thresh, score))
    if score > best_score:
        best_score = score
        best_thresh = thresh
print("All Pred best F1 score at threshold {0} is {1}".format(best_thresh, f1_score(y_true, (y_pred>best_thresh).astype(int))))

All Pred best F1 score at threshold 0.4 is 0.8537795505399359


In [24]:
test = pd.read_csv(os.path.join(INPUT_DIR,'test.csv'))
sub = pd.read_csv(os.path.join(INPUT_DIR,'sample_submit.csv'),header=None)
sub.columns = ['id','state']

In [25]:
def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub('',text)

def remove_html(text):
    html=re.compile(r"<[^>]*?>")
    return html.sub('',text)

def cleaning(texts):
    clean_texts = []
    for text in texts:
        # htmlタグを削除
        text = remove_URL(text)
        text = remove_html(text)
        #アルファベット以外をスペースに置き換え
        #clean_punc = re.sub(r'[^a-zA-Z]', ' ', text)
        #改行削除
        #text = text.replace("\n","")
        clean_texts.append(text)
    return clean_texts

def get_goal_values(df):
  df["goal"].replace("100000+","100000-100000",inplace=True)
  _df = df["goal"].str.split('-').apply(pd.Series).astype(float)
  _df.columns = ["goal_max","goal_min"]
  df["goal_max"] = _df["goal_max"].astype(str)
  df["goal_min"] = _df["goal_min"].astype(str)
  df["goal_median"] = _df[["goal_max","goal_min"]].median(axis=1)
  df["goal_median"] = df["goal_median"].astype(int)
  return df

if CFG20.clean_content==True:
    test['html_content'] = test['html_content'].map(lambda x: str(x))
    test['html_content'] = test['html_content'].apply(html.unescape)
    p = re.compile(r"<[^>]*?>|&amp;|[/'’\"”]")
    test['html_content'] = test['html_content'].map(lambda x: p.sub("", x))
    test['html_content'] = test['html_content'].map(lambda x: x.lstrip())
    test['html_content'] = test['html_content'].fillna('missing')

test = get_goal_values(test)
test['inputs1'] = test.goal_median.astype(str) + ' [SEP] ' + test.duration.astype(str) + ' [SEP] ' + test.country + ' [SEP] ' + test.category1 + ' [SEP] ' + test.category2 + ' [SEP] ' + test.html_content
test['inputs2'] = test.goal_median.astype(str) + ' [SEP] ' + test.duration.astype(str) + ' [SEP] ' + test.country + ' [SEP] ' + test.category2 + ' [SEP] ' + test.html_content

In [26]:
# ====================================================
# tokenizer
# ====================================================
#CFG19.tokenizer = AutoTokenizer.from_pretrained(CFG19.path+'tokenizer/')
CFG20.tokenizer = AutoTokenizer.from_pretrained(CFG20.path+'tokenizer/')
#CFG22.tokenizer = AutoTokenizer.from_pretrained(CFG22.path+'tokenizer/')
#CFG23.tokenizer = AutoTokenizer.from_pretrained(CFG23.path+'tokenizer/')
#CFG24.tokenizer = AutoTokenizer.from_pretrained(CFG24.path+'tokenizer/')
#CFG25.tokenizer = AutoTokenizer.from_pretrained(CFG25.path+'tokenizer/')
#CFG27.tokenizer = AutoTokenizer.from_pretrained(CFG27.path+'tokenizer/')
#CFG28.tokenizer = AutoTokenizer.from_pretrained(CFG28.path+'tokenizer/')
#CFG31.tokenizer = AutoTokenizer.from_pretrained(CFG31.path+'tokenizer/')
#CFG41.tokenizer = AutoTokenizer.from_pretrained(CFG41.path+'tokenizer/')
CFG42.tokenizer = AutoTokenizer.from_pretrained(CFG42.path+'tokenizer/')

#CFG21.tokenizer = AutoTokenizer.from_pretrained(CFG21.path+'tokenizer/')
#CFG43.tokenizer = AutoTokenizer.from_pretrained(CFG43.path+'tokenizer/')
CFG44.tokenizer = AutoTokenizer.from_pretrained(CFG44.path+'tokenizer/')
CFG45.tokenizer = AutoTokenizer.from_pretrained(CFG45.path+'tokenizer/')
#CFG46.tokenizer = AutoTokenizer.from_pretrained(CFG46.path+'tokenizer/')

CFG34.tokenizer = AutoTokenizer.from_pretrained(CFG34.path+'tokenizer/')
#CFG39.tokenizer = AutoTokenizer.from_pretrained(CFG39.path+'tokenizer/')
#CFG40.tokenizer = AutoTokenizer.from_pretrained(CFG40.path+'tokenizer/')
CFG47.tokenizer = AutoTokenizer.from_pretrained(CFG47.path+'tokenizer/')
CFG48.tokenizer = AutoTokenizer.from_pretrained(CFG48.path+'tokenizer/')

In [27]:
# ====================================================
# Dataset
# ====================================================
def prepare_input(cfg, text):
    inputs = cfg.tokenizer(text,
                           add_special_tokens=True,
                           max_length=cfg.max_len,
                           padding="max_length",
                           return_offsets_mapping=False,
                           truncation=True)
    for k, v in inputs.items():
        inputs[k] = torch.tensor(v, dtype=torch.long)
    return inputs


class TestDataset(Dataset):
    def __init__(self, cfg, df):
        self.cfg = cfg
        if cfg.target2col:
          self.inputs = df['inputs2'].values
        else:
          self.inputs = df['inputs1'].values

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, item):
        inputs = prepare_input(self.cfg, self.inputs[item])
        return inputs

In [28]:
# ====================================================
# Model
# ====================================================
class MeanPooling(nn.Module):
    def __init__(self):
        super(MeanPooling, self).__init__()
        
    def forward(self, last_hidden_state, attention_mask):
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
        sum_mask = input_mask_expanded.sum(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9)
        mean_embeddings = sum_embeddings / sum_mask
        return mean_embeddings

class MaxPooling(nn.Module):
    def __init__(self):
        super(MaxPooling, self).__init__()
        
    def forward(self, last_hidden_state, attention_mask):
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        embeddings = last_hidden_state.clone()
        embeddings[input_mask_expanded == 0] = -1e4
        max_embeddings, _ = torch.max(embeddings, dim=1)
        return max_embeddings
    

class CustomModel1(nn.Module):
    def __init__(self, cfg, config_path=None, pretrained=False):
        super().__init__()
        self.cfg = cfg
        if config_path is None:
            self.config = AutoConfig.from_pretrained(cfg.model, output_hidden_states=True)
            self.config.hidden_dropout = 0.
            self.config.hidden_dropout_prob = 0.
            self.config.attention_dropout = 0.
            self.config.attention_probs_dropout_prob = 0.
            LOGGER.info(self.config)
        else:
            self.config = torch.load(config_path)
        if pretrained:
            self.model = AutoModel.from_pretrained(cfg.model, config=self.config)
        else:
            self.model = AutoModel.from_config(self.config)
        if self.cfg.gradient_checkpointing:
            self.model.gradient_checkpointing_enable()

        # Freezing
        if cfg.freezing:
            # freezing embeddings and first 2 layers of encoder
            freeze((self.model).embeddings)
            freeze((self.model).encoder.layer[:2])
            cfg.after_freezed_parameters = filter(lambda parameter: parameter.requires_grad, (self.model).parameters())

        self.pool = MeanPooling()
        self.fc = nn.Linear(self.config.hidden_size, cfg.target_size)
        self._init_weights(self.fc)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
        
    def feature(self, inputs):
        outputs = self.model(**inputs)
        last_hidden_states = outputs[0]
        feature = self.pool(last_hidden_states, inputs['attention_mask'])
        return feature

    def forward(self, inputs):
        feature = self.feature(inputs)
        output = self.fc(feature)
        return output


class CustomModel2(nn.Module):
    def __init__(self, cfg, config_path=None, pretrained=False):
        super().__init__()
        self.cfg = cfg
        if config_path is None:
            self.config = AutoConfig.from_pretrained(cfg.model, output_hidden_states=True)
            self.config.hidden_dropout = 0.
            self.config.hidden_dropout_prob = 0.
            self.config.attention_dropout = 0.
            self.config.attention_probs_dropout_prob = 0.
            LOGGER.info(self.config)
        else:
            self.config = torch.load(config_path)
        if pretrained:
            self.model = AutoModel.from_pretrained(cfg.model, config=self.config)
        else:
            self.model = AutoModel.from_config(self.config)
        if self.cfg.gradient_checkpointing:
            self.model.gradient_checkpointing_enable()

        # Freezing
        if cfg.freezing:
            # freezing embeddings and first 2 layers of encoder
            freeze((self.model).embeddings)
            freeze((self.model).encoder.layer[:2])
            cfg.after_freezed_parameters = filter(lambda parameter: parameter.requires_grad, (self.model).parameters())

        self.pool = MeanPooling()
        self.fc = nn.Linear(self.config.hidden_size, cfg.target_size)
        #self._init_weights(self.fc)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
        
    def feature(self, inputs):
        outputs = self.model(**inputs)
        last_hidden_states = outputs[0]
        feature = self.pool(last_hidden_states, inputs['attention_mask'])
        return feature

    def forward(self, inputs):
        feature = self.feature(inputs)
        output = self.fc(feature)
        return output

class CustomModel3(nn.Module):
    def __init__(self, cfg, config_path=None, pretrained=False):
        super().__init__()
        self.cfg = cfg
        if config_path is None:
            self.config = AutoConfig.from_pretrained(cfg.model, output_hidden_states=True)
            self.config.hidden_dropout = 0.
            self.config.hidden_dropout_prob = 0.
            self.config.attention_dropout = 0.
            self.config.attention_probs_dropout_prob = 0.
            LOGGER.info(self.config)
        else:
            self.config = torch.load(config_path)
        if pretrained:
            self.model = AutoModel.from_pretrained(cfg.model, config=self.config)
        else:
            self.model = AutoModel.from_config(self.config)
        if self.cfg.gradient_checkpointing:
            self.model.gradient_checkpointing_enable()

        # Freezing
        if cfg.freezing:
            # freezing embeddings and first 2 layers of encoder
            freeze((self.model).embeddings)
            freeze((self.model).encoder.layer[:2])
            cfg.after_freezed_parameters = filter(lambda parameter: parameter.requires_grad, (self.model).parameters())

        #self.pool = MeanPooling()
        self.fc_dropout = nn.Dropout(cfg.fc_dropout)
        self.fc = nn.Linear(self.config.hidden_size, cfg.target_size)
        self._init_weights(self.fc)
        self.attention = nn.Sequential(
            nn.Linear(self.config.hidden_size, 512),
            nn.Tanh(),
            nn.Linear(512, 1),
            nn.Softmax(dim=1)
        )
        self._init_weights(self.attention)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
        
    def feature(self, inputs):
        outputs = self.model(**inputs)
        last_hidden_states = outputs[0]
        weights = self.attention(last_hidden_states)
        feature = torch.sum(weights * last_hidden_states, dim=1)
        return feature

    def forward(self, inputs):
        feature = self.feature(inputs)
        output = self.fc(self.fc_dropout(feature))
        return output

class CustomModel4(nn.Module):
    def __init__(self, cfg, config_path=None, pretrained=False):
        super().__init__()
        self.cfg = cfg
        if config_path is None:
            self.config = AutoConfig.from_pretrained(cfg.model, output_hidden_states=True)
            self.config.hidden_dropout = 0.
            self.config.hidden_dropout_prob = 0.
            self.config.attention_dropout = 0.
            self.config.attention_probs_dropout_prob = 0.
            LOGGER.info(self.config)
        else:
            self.config = torch.load(config_path)
        if pretrained:
            self.model = AutoModel.from_pretrained(cfg.model, config=self.config)
        else:
            self.model = AutoModel.from_config(self.config)
        if self.cfg.gradient_checkpointing:
            self.model.gradient_checkpointing_enable()

        # Freezing
        if cfg.freezing:
            # freezing embeddings and first 2 layers of encoder
            freeze((self.model).embeddings)
            freeze((self.model).encoder.layer[:2])
            cfg.after_freezed_parameters = filter(lambda parameter: parameter.requires_grad, (self.model).parameters())

        #self.pool = MeanPooling()
        self.fc_dropout = nn.Dropout(cfg.fc_dropout)
        self.fc = nn.Linear(self.config.hidden_size, cfg.target_size)
        #self._init_weights(self.fc)
        self.attention = nn.Sequential(
            nn.Linear(self.config.hidden_size, 512),
            nn.Tanh(),
            nn.Linear(512, 1),
            nn.Softmax(dim=1)
        )
        #self._init_weights(self.attention)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
        
    def feature(self, inputs):
        outputs = self.model(**inputs)
        last_hidden_states = outputs[0]
        weights = self.attention(last_hidden_states)
        feature = torch.sum(weights * last_hidden_states, dim=1)
        return feature

    def forward(self, inputs):
        feature = self.feature(inputs)
        output = self.fc(self.fc_dropout(feature))
        return output

In [29]:
# ====================================================
# inference
# ====================================================
def inference_fn(test_loader, model, device):
    preds = []
    model.eval()
    model.to(device)
    tk0 = tqdm(test_loader, total=len(test_loader))
    for inputs in tk0:
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        with torch.no_grad():
            y_preds = model(inputs)
        preds.append(y_preds.sigmoid().to('cpu').numpy())
    predictions = np.concatenate(preds)
    return predictions

In [30]:
test_dataset20 = TestDataset(CFG20, test)
test_loader20 = DataLoader(test_dataset20,
                         batch_size=CFG20.batch_size,
                         shuffle=False,
                         num_workers=CFG20.num_workers, pin_memory=True, drop_last=False)
predictions_20 = []
for fold in CFG20.trn_fold:
    model = CustomModel1(CFG20, config_path=CFG20.config_path, pretrained=False)
    state = torch.load(CFG20.path+f"{CFG20.model.replace('/', '-')}_fold{fold}_best.pth",
                       map_location=torch.device('cpu'))
    model.load_state_dict(state['model'])
    prediction = inference_fn(test_loader20, model, device)
    predictions_20.append(prediction)
    del model, state, prediction; gc.collect()
    torch.cuda.empty_cache()
predictions_20 = np.mean(predictions_20, axis=0)

 25%|██▌       | 78/307 [2:28:17<7:16:41, 114.42s/it]

In [31]:
test_dataset42 = TestDataset(CFG42, test)
test_loader42 = DataLoader(test_dataset42,
                         batch_size=CFG42.batch_size,
                         shuffle=False,
                         num_workers=CFG42.num_workers, pin_memory=True, drop_last=False)
predictions_42 = []
for fold in CFG42.trn_fold:
    model = CustomModel2(CFG42, config_path=CFG42.config_path, pretrained=False)
    state = torch.load(CFG42.path+f"{CFG42.model.replace('/', '-')}_fold{fold}_best.pth",
                       map_location=torch.device('cpu'))
    model.load_state_dict(state['model'])
    prediction = inference_fn(test_loader42, model, device)
    predictions_42.append(prediction)
    del model, state, prediction; gc.collect()
    torch.cuda.empty_cache()
predictions_42 = np.mean(predictions_42, axis=0)

100%|██████████| 307/307 [06:13<00:00,  1.22s/it]


In [32]:
test_dataset44 = TestDataset(CFG44, test)
test_loader44 = DataLoader(test_dataset44,
                         batch_size=CFG44.batch_size,
                         shuffle=False,
                         num_workers=CFG44.num_workers, pin_memory=True, drop_last=False)
predictions_44 = []
for fold in CFG44.trn_fold:
    model = CustomModel3(CFG44, config_path=CFG44.config_path, pretrained=False)
    state = torch.load(CFG44.path+f"{CFG44.model.replace('/', '-')}_fold{fold}_best.pth",
                       map_location=torch.device('cpu'))
    model.load_state_dict(state['model'])
    prediction = inference_fn(test_loader44, model, device)
    predictions_44.append(prediction)
    del model, state, prediction; gc.collect()
    torch.cuda.empty_cache()
predictions_44 = np.mean(predictions_44, axis=0)

 31%|███▏      | 96/307 [03:12<07:03,  2.01s/it]


KeyboardInterrupt: ignored

In [ ]:
test_dataset45 = TestDataset(CFG45, test)
test_loader45 = DataLoader(test_dataset45,
                         batch_size=CFG45.batch_size,
                         shuffle=False,
                         num_workers=CFG45.num_workers, pin_memory=True, drop_last=False)
predictions_45 = []
for fold in CFG45.trn_fold:
    model = CustomModel4(CFG45, config_path=CFG45.config_path, pretrained=False)
    state = torch.load(CFG45.path+f"{CFG45.model.replace('/', '-')}_fold{fold}_best.pth",
                       map_location=torch.device('cpu'))
    model.load_state_dict(state['model'])
    prediction = inference_fn(test_loader45, model, device)
    predictions_45.append(prediction)
    del model, state, prediction; gc.collect()
    torch.cuda.empty_cache()
predictions_45 = np.mean(predictions_45, axis=0)

In [ ]:
test_dataset34 = TestDataset(CFG34, test)
test_loader34 = DataLoader(test_dataset34,
                         batch_size=CFG34.batch_size,
                         shuffle=False,
                         num_workers=CFG34.num_workers, pin_memory=True, drop_last=False)
predictions_34 = []
for fold in CFG34.trn_fold:
    model = CustomModel1(CFG34, config_path=CFG34.config_path, pretrained=False)
    state = torch.load(CFG34.path+f"{CFG34.model.replace('/', '-')}_fold{fold}_best.pth",
                       map_location=torch.device('cpu'))
    model.load_state_dict(state['model'])
    prediction = inference_fn(test_loader34, model, device)
    predictions_34.append(prediction)
    del model, state, prediction; gc.collect()
    torch.cuda.empty_cache()
predictions_34 = np.mean(predictions_34, axis=0)
predictions_34 = predictions_34[:,0]
predictions_34 = predictions_34.reshape((-1,1))

In [ ]:
test_dataset47 = TestDataset(CFG47, test)
test_loader47 = DataLoader(test_dataset47,
                         batch_size=CFG47.batch_size,
                         shuffle=False,
                         num_workers=CFG47.num_workers, pin_memory=True, drop_last=False)
predictions_47 = []
for fold in CFG47.trn_fold:
    model = CustomModel1(CFG47, config_path=CFG47.config_path, pretrained=False)
    state = torch.load(CFG47.path+f"{CFG47.model.replace('/', '-')}_fold{fold}_best.pth",
                       map_location=torch.device('cpu'))
    model.load_state_dict(state['model'])
    prediction = inference_fn(test_loader47, model, device)
    predictions_47.append(prediction)
    del model, state, prediction; gc.collect()
    torch.cuda.empty_cache()
predictions_47 = np.mean(predictions_47, axis=0)
predictions_47 = predictions_47[:,0]
predictions_47 = predictions_47.reshape((-1,1))

In [ ]:
test_dataset48 = TestDataset(CFG48, test)
test_loader48 = DataLoader(test_dataset48,
                         batch_size=CFG48.batch_size,
                         shuffle=False,
                         num_workers=CFG48.num_workers, pin_memory=True, drop_last=False)
predictions_48 = []
for fold in CFG48.trn_fold:
    model = CustomModel2(CFG48, config_path=CFG48.config_path, pretrained=False)
    state = torch.load(CFG48.path+f"{CFG48.model.replace('/', '-')}_fold{fold}_best.pth",
                       map_location=torch.device('cpu'))
    model.load_state_dict(state['model'])
    prediction = inference_fn(test_loader48, model, device)
    predictions_48.append(prediction)
    del model, state, prediction; gc.collect()
    torch.cuda.empty_cache()
predictions_48 = np.mean(predictions_48, axis=0)
predictions_48 = predictions_48[:,0]
predictions_48 = predictions_48.reshape((-1,1))

In [ ]:
stacking_test = sub.copy()
#stacking_test["preds_19"] = predictions_19
stacking_test["preds_20"] = predictions_20
#stacking_test["preds_22"] = predictions_22
#stacking_test["preds_23"] = predictions_23
#stacking_test["preds_24"] = predictions_24
#stacking_test["preds_25"] = predictions_25
#stacking_test["preds_27"] = predictions_27
#stacking_test["preds_28"] = predictions_28
#stacking_test["preds_31"] = predictions_31
#stacking_test["preds_41"] = predictions_41
stacking_test["preds_42"] = predictions_42

#stacking_test["preds_21"] = predictions_21
#stacking_test["preds_43"] = predictions_43
stacking_test["preds_44"] = predictions_44
stacking_test["preds_45"] = predictions_45
#stacking_test["preds_46"] = predictions_46

stacking_test["preds_34"] = predictions_34
#stacking_test["preds_39"] = predictions_39
#stacking_test["preds_40"] = predictions_40
stacking_test["preds_47"] = predictions_47
stacking_test["preds_48"] = predictions_48

stacking_test["preds_max"] = stacking_test.loc[:,"preds_20":"preds_48"].max(axis=1)
stacking_test["preds_min"] = stacking_test.loc[:,"preds_20":"preds_48"].min(axis=1)
stacking_test["preds_mean"] = stacking_test.loc[:,"preds_20":"preds_48"].mean(axis=1)
stacking_test["preds_std"] = stacking_test.loc[:,"preds_20":"preds_48"].std(axis=1)

stacking_test = stacking_test.merge(test_df,how="left",on=["id"])

In [ ]:
def get_predicts(df_,lgb_clfs,ctb_clfs,logit_clfs,svm_clfs):
  lgb_predict,ctb_predict,logit_predict,svm_predict = [],[],[],[]
  test_X = df_[features].values
  for fold in range(4):
    lgb_pred = lgb_clfs[fold].predict_proba(test_X)
    lgb_pred = lgb_pred[:,1]
    lgb_predict.append(lgb_pred)
    ctb_pred = ctb_clfs[fold].predict_proba(test_X)
    ctb_pred = ctb_pred[:,1]
    ctb_predict.append(ctb_pred)

  return lgb_predict,ctb_predict

lgb_preds,ctb_preds = get_predicts(stacking_test,lgbm_clf_,ctb_clf_,logit_clf_,svm_clf_)

In [ ]:
lgb_preds = np.mean(lgb_preds,axis=0)
ctb_preds = np.mean(ctb_preds,axis=0)


#predictions = lgb_pred*1.40719207e-01 + ctb_pred*3.79171486e-02 + logit_pred*3.99166372e-05 + svm_pred*8.21323728e-01
predictions = (lgb_preds+ctb_preds) / 2

In [ ]:
submit = sub.copy()
#submit.columns = ["id","label"]

submit['label'] = predictions
submit['state'] = (predictions>0.5).astype(int)
submit['best_state'] = (predictions>best_thresh).astype(int)

submit[["id","state"]].to_csv(os.path.join(OUTPUT_SUB_DIR,"stacking_submission8477.csv"),index=False,header=False)
submit[["id","best_state"]].to_csv(os.path.join(OUTPUT_SUB_DIR,"stacking_bestthresh_submission8510.csv"),index=False,header=False)
display(submit)

In [ ]:
plt.hist(submit.label,bins=25,edgecolor='white')
plt.xlabel("Pred")
plt.show()

In [ ]:
#train,testでダブっているデータを参照
dup_test_ids = ['test_01704','test_03707','test_04353','test_04453','test_04645','test_06909','test_08232']
sub_dup= submit[submit["id"].isin(dup_test_ids)]

T = pd.DataFrame([['train_00365','train_01463','train_02156','train_04506','train_06891','train_08302','train_09470'],
                [0,1,1,1,1,1,0],
                ['test_08232','test_01704','test_06909','test_04453','test_03707','test_04353','test_04645']]).T
T.columns =["train id","true label","id"]
T = T.sort_values(by="id")
T = T.merge(sub_dup,how='left',on='id')
T